In [1]:
"""
Solved:
    -It's possible for train-test split to split data in such a way, that
   after encoding, X_train and X_test have different numbers of features.
   Split has to be rerun to fix it. First encode, then split again?
   BUT IT STILL HAS TO BE ENCODED AND SPLIT BEFORE STARTING CROSS VALIDATION
   AND SEQUENTIAL FEATURE SELECTION. MAYBE APPEND DURING SPLIT AND THEN SPLIT
   AGAIN?
    -Improve feature encoding to have proper ordering instead of random numbers
    which currently influence classification accuracy:
    https://datascience.stackexchange.com/questions/72343/encoding-with-ordinalencoder-how-to-give-levels-as-user-input

Fishy:
    -check and check for data leakage (def: https://scikit-learn.org/stable/glossary.html)

Pickup: 
    -that function for gathering columns to encode and putting that into encoders
    -wrap encoding and classifier into a class and call it EncodingCategoricalNaiveBayes
TODO:
    -put the classifier in SFS
    -modify SFS to check each predict_proba and stop if threshold is hit
    -add cost counting to SFS wrapper
    -???
    -tests and profit???

SyntaxError: EOF while scanning triple-quoted string literal (Temp/ipykernel_1480/1084367042.py, line 18)

In [2]:
## SKLEARN
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.compose import make_column_transformer

import numpy as np

np.set_printoptions(suppress=True)
import pandas as pd
from platform import python_version

import scipy

import random

# works, sort of only. Possible additional commas that shouldn't be there.
%load_ext nb_black

print("Libs imported. Python version is: ", python_version())

Libs imported. Python version is:  3.9.7


<IPython.core.display.Javascript object>

In [3]:
# utility functions

cols_mushroom = [
    "labels",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat",
]

cols_car = ["buying", "maintenance", "doors", "passengers", "boot", "safety", "labels"]

cols_audiology = [
    "age_gt_60",
    "air",
    "airBoneGap",
    "ar_c",
    "ar_u",
    "bone",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "o_ar_c",
    "o_ar_u",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "speech",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
    "p-index",
    "labels",
]

cols_wine = [
    "labels",
    "alcohol",
    "mallic-acid",
    "alcalinity",
    "ash",
    "magnesium",
    "total-phenols",
    "flavonids",
    "nonflavonid-phenols",
    "proanthocyanins",
    "color-intensity",
    "hue",
    "od-of-diluted-wines",
    "proline",
]

"""
https://archive.ics.uci.edu/ml/datasets/car+evaluation
0-5 -> data
6 -> labels
"""


def load_car():
    df_car = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
        header=None,
        names=cols_car,
    )
    # mappings using indexes:
    # X = df_car.loc[:, :5].values
    # y = df_car.loc[:, 6].values
    labels_col = df_car.pop("labels")
    df_car.insert(0, "labels", labels_col)
    # replace 5more in doors to 5
    # df_car.loc[df_car['doors'] == '5more', 'doors'] = '5'
    # df_car["doors"] = pd.to_numeric(df_car["doors"])
    # replace more in passengers to 5
    return df_car


"""
https://archive.ics.uci.edu/ml/datasets/mushroom
1-22 -> data
0 -> labels
"""


def load_mushroom():
    df_mushroom = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
        header=None,
        names=cols_mushroom,
    )
    # index mappings
    # X = df_mushroom.loc[:, 1:].values
    # y = df_mushroom.loc[:, 0].values
    return df_mushroom


"""
https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29
0:length-2 -> data
length-1 unique id (p1-p200)
length -> labels
"""


def load_audiology():
    df_audiology = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/audiology/audiology.standardized.data",
        header=None,
        names=cols_audiology,
    )
    # index mapping
    # length = len(df_audiology.columns)
    # X = df_audiology.loc[:, : length - 3].values
    # y = df_audiology.loc[:, length - 1].values
    df_audiology = df_audiology.drop("p-index", axis=1)
    labels_col = df_audiology.pop("labels")
    df_audiology.insert(0, "labels", labels_col)
    return df_audiology


"""
https://www.alldatascience.com/classification/wine-dataset-analysis-with-python/
1:length -> data
0 -> labels
"""


def load_wine():
    df_wine = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",
        header=None,
        names=cols_wine,
    )
    # index mappings
    # length = len(df_wine.columns)
    # X = df_wine.loc[:, 1:].values
    # y = df_wine.loc[:, 0].values
    return df_wine

<IPython.core.display.Javascript object>

In [12]:
# Choose dataset
# dataset = load_car()
dataset = load_mushroom()
# dataset = load_audiology()
### dataset = load_wine() # all cols numerical, doesn't work

print(dataset.info())
# print("First five records:")
# print(dataset.head())

# Extract to X and y
X_cat = dataset.loc[:, dataset.columns != "labels"]
y_cat = dataset.loc[:, "labels"]

print("Size of X: ", np.shape(X_cat))
print("Size of y: ", np.shape(y_cat))

# Generate a matrix of costs
max_cost_allowed = 10000

dataset_costs = pd.DataFrame(
    np.random.randint(0, max_cost_allowed, size=(1, np.shape(X_cat)[1])),
    columns=X_cat.columns,
)

print("Size of dataset costs: ", np.shape(dataset_costs))
print("Cost of classification on full dataset: ", dataset_costs.sum(axis=1)[0])

max_seed_val = 2 ** 32 - 1

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X_cat, y_cat, test_size=0.2, random_state=random.randrange(0, max_seed_val),
)
print("Data has been split.")
# print("X contains features: ", X_train.columns == "index")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   labels                    8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

<IPython.core.display.Javascript object>

In [5]:
# Collectors of values

cols_one_hot = [
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "habitat",
    "age_gt_60",
    "airBoneGap",
    "boneAbnormal",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "static_normal",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
]

cols_ordinal = [
    "buying",
    "maintenance",
    "doors",
    "passengers",
    "boot",
    "safety",
    "population",
    "air",
    "ar_c",
    "ar_u",
    "bser",
    "bone",
    "o_ar_c",
    "o_ar_u",
    "speech",
    "tymp",
]


def intersection(lst1, lst2):
    return [value for value in lst1 if value in lst2]


def collect_current_one_hot_columns(argNames):
    # make list of all values and create steps for them
    return intersection(cols_one_hot, argNames)


def collect_current_ordinal_columns(argNames):
    # make list of all values and create steps for them
    return intersection(cols_ordinal, argNames)

<IPython.core.display.Javascript object>

In [6]:
# Make order of categories per each column in ordinal_columns
order_of_ordinal_categories = pd.DataFrame.from_dict(
    {
        "buying": ["low", "med", "high", "vhigh", "filler1", "filler2", "filler3"],
        "maintenance": ["low", "med", "high", "vhigh", "filler1", "filler2", "filler3"],
        "doors": ["2", "3", "4", "5more", "filler1", "filler2", "filler3"],
        "passengers": ["2", "4", "more", "filler1", "filler2", "filler3", "filler4"],
        "boot": ["small", "med", "big", "filler1", "filler2", "filler3", "filler4"],
        "safety": ["low", "med", "high", "filler1", "filler2", "filler3", "filler4"],
        "population": ["y", "v", "s", "n", "c", "a", "filler1"],
        "air": [
            "normal",
            "mild",
            "moderate",
            "severe",
            "profound",
            "filler1",
            "filler2",
        ],
        "ar_c": ["?", "absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "ar_u": ["?", "absent", "normal", "elevated", "filler1", "filler2", "filler3"],
        "bser": ["?", "normal", "degraded", "filler1", "filler2", "filler3", "filler4"],
        "bone": ["?", "unmeasured", "normal", "mild", "moderate", "filler1", "filler3"],
        "o_ar_c": [
            "?",
            "absent",
            "normal",
            "elevated",
            "filler1",
            "filler2",
            "filler3",
        ],
        "o_ar_u": [
            "?",
            "absent",
            "normal",
            "elevated",
            "filler1",
            "filler2",
            "filler3",
        ],
        "speech": [
            "?",
            "unmeasured",
            "very_poor",
            "poor",
            "normal",
            "good",
            "very_good",
        ],
        "tymp": ["a", "as", "b", "ad", "c", "filler1", "filler2"],
    }
)

print("Order created.")
print(order_of_ordinal_categories)

Order created.
    buying maintenance    doors passengers     boot   safety population  \
0      low         low        2          2    small      low          y   
1      med         med        3          4      med      med          v   
2     high        high        4       more      big     high          s   
3    vhigh       vhigh    5more    filler1  filler1  filler1          n   
4  filler1     filler1  filler1    filler2  filler2  filler2          c   
5  filler2     filler2  filler2    filler3  filler3  filler3          a   
6  filler3     filler3  filler3    filler4  filler4  filler4    filler1   

        air      ar_c      ar_u      bser        bone    o_ar_c    o_ar_u  \
0    normal         ?         ?         ?           ?         ?         ?   
1      mild    absent    absent    normal  unmeasured    absent    absent   
2  moderate    normal    normal  degraded      normal    normal    normal   
3    severe  elevated  elevated   filler1        mild  elevated  elevated   

<IPython.core.display.Javascript object>

In [17]:
def calculate_current_order_of_ordinal_columns_to_encode():
    # Get common cols to feed them in proper order to ordinal encoder
    index_of_common_cols = order_of_ordinal_categories.columns.intersection(
        current_columns_ordinal
    )
    # Convert to list
    order_of_ordinal_categories_list = (
        order_of_ordinal_categories[index_of_common_cols].values.transpose().tolist()
    )
    return order_of_ordinal_categories_list


# Get current ordinal and one hot columns
total_column_list = X_cat.select_dtypes(include="object").columns
current_columns_one_hot = collect_current_one_hot_columns(total_column_list)
current_columns_ordinal = collect_current_ordinal_columns(total_column_list)

current_ordinal_col_ordering_to_encode = (
    calculate_current_order_of_ordinal_columns_to_encode()
)

print("one hot: ", current_columns_one_hot)
print("ordinal: ", current_columns_ordinal)

# for debugging: unknown value in n-th column means n-th col from this is missing that value(s)
# print(index_of_common_cols)
# for col in X_train:
#    print(col, ": ", X_train[col].unique())

# Create column transformer
column_transform = make_column_transformer(
    (OneHotEncoder(), current_columns_one_hot),
    (
        OrdinalEncoder(categories=current_ordinal_col_ordering_to_encode),
        current_columns_ordinal,
    ),
    n_jobs=1,
)

# Apply transformation
column_transform.fit(X_cat)
encoded_X_train = column_transform.transform(X_train)
encoded_X_test = column_transform.transform(X_test)
print("Successfully encoded dataset.")

if scipy.sparse.issparse(encoded_X_train):
    encoded_X_train = encoded_X_train.toarray()
if scipy.sparse.issparse(encoded_X_test):
    encoded_X_test = encoded_X_test.toarray()

print("X_train", np.shape(encoded_X_train))
print(encoded_X_train[0:5])
print(np.shape(encoded_X_test))
print(encoded_X_test[0:5])

# Transform y using label encoder
le = LabelEncoder().fit(y_cat)
encoded_y_train = le.transform(y_train)
encoded_y_test = le.transform(y_test)
print("Successfully encoded dataset labels.")

print(
    "Train dataset contains ",
    encoded_X_train.shape,
    ", y dataset contains ",
    encoded_y_train.shape,
)
print(
    "Test dataset contains ",
    encoded_X_test.shape,
    ", y dataset contains ",
    encoded_y_test.shape,
)

one hot:  ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'habitat']
ordinal:  ['population']
Successfully encoded dataset.
X_train (6499, 112)
[[0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.

<IPython.core.display.Javascript object>

In [20]:
# Create a Bayes Classifier
nbayes = CategoricalNB()

# Train the model using the training sets
nbayes.fit(encoded_X_train, y_train)

# Predict the response for test dataset
y_pred = nbayes.predict(encoded_X_test)
y_pred_probas = nbayes.predict_proba(encoded_X_test)
print(y_pred_probas[0:10])

[[0.         1.        ]
 [0.99999985 0.00000015]
 [0.         1.        ]
 [0.99999997 0.00000003]
 [0.00001758 0.99998242]
 [1.         0.        ]
 [0.85114924 0.14885076]
 [1.         0.        ]
 [0.00594469 0.99405531]
 [0.         1.        ]]


<IPython.core.display.Javascript object>

In [21]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred) * 100, "%")
print("F1 score:", metrics.f1_score(y_test, y_pred, average="weighted") * 100, "%")

Accuracy: 94.15384615384616 %
F1 score: 94.13533681411134 %


<IPython.core.display.Javascript object>

In [22]:
# Get cost of classification
classification_cost = dataset_costs[X_cat.columns].sum(axis=1)[0]
print(classification_cost)

137593


<IPython.core.display.Javascript object>

In [ ]:
"""
useful links:

https://dev.to/codinghappinessweb/analysing-dataset-using-naive-bayes-classifier-3d7o
https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c
https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79
https://towardsdatascience.com/naive-bayes-classifier-how-to-successfully-use-it-in-python-ecf76a995069
https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
"""